# IMPORT STATMENTS FOR ALL MODLES

In [1]:
!TMPDIR=/home/mgill/ pip install --cache-dir=/home/mgill/ --build /home/mgill/ pandas numpy sklearn xgboost matplotlib tensorflow==2.1.0 keras==2.2.4 fastai python-dateutil scikit-optimize scikit-learn==0.21 seaborn

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [2]:
import tensorflow as tf; print(tf.__version__)
import keras; print(keras.__version__)
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.regularizers import *
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from fastai.tabular.all import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from random import randint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Activation
from math import sqrt
from statistics import mean
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt 
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.impute import SimpleImputer
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from keras.optimizers import Adam
import skopt
from skopt.searchcv import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import GridSearchCV
import pickle
from collections import OrderedDict
from sklearn.model_selection import StratifiedKFold
from scipy import interp
import seaborn as sns
sns.set()
sns.set_style('whitegrid')

2.1.0


Using TensorFlow backend.


2.2.4


In [3]:
def new_prep_data(tt_file, ho_file):
    imp = SimpleImputer(missing_values='./.', strategy='most_frequent')
    my_list = []
    x = 0 
    for chunk in pd.read_csv(tt_file, chunksize=10000, index_col="Unnamed: 0"):
        x=x+10000
        chunk = chunk.T
        if 'Value' in chunk.columns:
            #does the selecting of pheno array for application ML
            chunk["Value"] = pd.to_numeric(chunk["Value"], downcast="float")
            tt_pheno = chunk["Value"].to_numpy()
            #reshapes it so its not a 1D array
            print(tt_pheno.shape)
            tt_pheno = np.reshape(tt_pheno,(len(tt_pheno),1))
            print(tt_pheno.shape)
            chunk = chunk.drop(columns=['Value'])
        headers = chunk.columns
        row_idx = chunk.index
        chunk = imp.fit_transform(chunk) #SHOULD TURN ./. into the most common for each column
        #since imputing makes a numpy array have to turn back into PD for label encoding
        chunk = pd.DataFrame(data = chunk, index = row_idx, columns = headers)
        my_list.append(chunk)
        print(x)
    tt_vcf = pd.concat(my_list, axis = 1)
    my_list = []
    x=0
    for chunk in pd.read_csv(ho_file, chunksize=10000, index_col="Unnamed: 0"):
        x=x+10000
        chunk = chunk.T
        if 'Value' in chunk.columns:
            #does the selecting of pheno array for application ML
            chunk["Value"] = pd.to_numeric(chunk["Value"], downcast="float")
            ho_pheno = chunk["Value"].to_numpy()
            #reshapes it so its not a 1D array
            print(ho_pheno.shape)
            ho_pheno = np.reshape(ho_pheno,(len(ho_pheno),1))
            print(ho_pheno.shape)
            chunk = chunk.drop(columns=['Value'])
        headers = chunk.columns
        row_idx = chunk.index
        chunk = imp.fit_transform(chunk) #SHOULD TURN ./. into the most common for each column
        #since imputing makes a numpy array have to turn back into PD for label encoding
        chunk = pd.DataFrame(data = chunk, index = row_idx, columns = headers)
        my_list.append(chunk)
        print(x)
    ho_vcf = pd.concat(my_list, axis = 1)
    print(tt_vcf.shape)
    print(ho_vcf.shape)
    return tt_vcf, ho_vcf, tt_pheno, ho_pheno

In [ ]:
tt_vcf, ho_vcf, tt_pheno, ho_pheno = new_prep_data("Pro_Merged_filtered.csv_train_test.csv_5pcnt.csv", "Pro_Merged_filtered.csv_holdout.csv_5pcnt.csv")

In [ ]:
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
ohe = ohe.fit(tt_vcf)
print(tt_vcf.shape)
tt_vcf = ohe.transform(tt_vcf)
print(tt_vcf.shape)
print(ho_vcf.shape)
ho_vcf = ohe.transform(ho_vcf)
print(ho_vcf.shape)

In [ ]:
pickle.dump(ohe, open("Pro_ohe.dat", "wb"))

In [ ]:
#if need or have new holdout data etc.
ohe = pickle.load(open("Pro_ohe.dat", "rb"))

In [ ]:
def find_snp_from_header(ohe,snp_num):
    count = 0
    snp = "Not found"
    found = False
    i = 0
    while i < len(ohe.categories_) and (found == False):
        j = 0
        while j < len(ohe.categories_[i]):
            if(count == snp_num):
                snp = ohe.categories_[i][j]
                found = True
                break
            count = count + 1
            j = j + 1
        i = i + 1
    return snp

In [ ]:
## TESTING IF IT WORKS
my_snp = find_snp_from_header(ohe, 108005)
print(my_snp)

# XGBoost ML Model

In [ ]:
print(tt_vcf.shape)
print(tt_pheno.shape)
seed = randint(0,5000)
X_train, X_test, y_train, y_test = train_test_split(tt_vcf, tt_pheno, test_size=0.2, random_state=seed)
print(X_test.shape)
print("seed is " + str(seed))

In [ ]:
space ={'learning_rate': Real(0.01, 1.0, 'log-uniform'),
        'min_child_weight': Integer(0, 10),
        'max_depth': Integer(0, 50),
        'max_delta_step': Integer(0, 20),
        'subsample': Real(0.01, 1.0, 'uniform'),
        'colsample_bytree': Real(0.01, 1.0, 'uniform'),
        'colsample_bylevel': Real(0.01, 1.0, 'uniform'),
        'reg_lambda': Real(1e-9, 1000, 'log-uniform'),
        'reg_alpha': Real(1e-9, 1.0, 'log-uniform'),
        'gamma': Real(1e-9, 0.5, 'log-uniform'),
        'min_child_weight': Integer(0, 5),
        'n_estimators': Integer(50, 200),
        'scale_pos_weight': Real(1e-6, 500, 'log-uniform')}


In [ ]:
def on_step(optim_result):
    """
    Callback meant to view scores after
    each iteration while performing Bayesian
    Optimization in Skopt"""
    score = xgb_bayes_search.best_score_
    print("best score: %s" % score)
    if score >= 0.98:
        print('Interrupting!')
        return True

In [ ]:
xgbreg = xgb.XGBRegressor()
xgb_bayes_search = BayesSearchCV(xgbreg, space, n_iter=32, # specify how many iterations
                                    scoring=None, n_jobs=1, cv=5, verbose=3, random_state=42, n_points=12,
                                 refit=True)
xgb_bayes_search.fit(X_train, y_train.ravel(), callback = on_step)
print(xgb_bayes_search.best_params_)

In [ ]:
#check to make sure all the data has remained the same
np.isnan(X_train).any()
np.isnan(y_train).any()

np.isinf(X_train).any()
np.isinf(y_train).any()

In [ ]:
xgb_bayes_search.best_params_

In [ ]:
##START HERE

#xgbreg = xgb_bayes_search.best_estimator_ #xgb.XGBRegressor(#n_estimators=100, nthread=-1) 
#best_params = xgb_bayes_search.best_params_

best_params = OrderedDict([('colsample_bylevel', 0.88542798001491),
             ('colsample_bytree', 0.15107725561517718),
             ('gamma', 0.09274068410384222),
             ('learning_rate', 0.11437510345794359),
             ('max_delta_step', 0),
             ('max_depth', 6),
             ('min_child_weight', 5),
             ('n_estimators', 200),
             ('reg_alpha', 1.2952579279714691e-08),
             ('reg_lambda', 0.45707493844512304),
             ('scale_pos_weight', 10.561837873121673),
             ('subsample', 0.9105119208713889)])
xgbreg = xgb.XGBRegressor(**best_params)

In [ ]:
print(xgbreg)

In [ ]:
def eval_k_fold(m, x, y, k):
    #model: xgboost model, should be with the best params available
    #x: input data (eg. all samples and SNPS)
    #y: labels
    #k: number of folds for cross validation
    cv = KFold(n_splits=k,shuffle=True)
  #  fig1 = plt.figure(figsize=[12,12])

   # tprs = []
   # aucs = []
    results = []
   # mean_fpr = np.linspace(0,1,100)
    low = 100
    best = m
    i = 1
    for train,test in cv.split(x,y):
        #print(y[test])
        m.fit(x[train],y[train].ravel())
        print("fitting done. Processing fold accuracy + checking best model")
        #predictions = [round(value) for value in y_pred]
        #sees how accurate the model was when testing the test set
        all_preds = [x for x in m.predict(x[test])]
        ss = sqrt(mean_squared_error(all_preds, y[test]))
        rr = r2_score(all_preds, y[test])
        mm = np.mean(y[test])
        error_mean = ((ss/mm)*100)
        print("R^2 Value is: " + str(rr))
        print("RMSE for dataset is:" +str(ss) + "& mean of this fold is " + str(mm))
        print("this is "+ str((ss/mm)*100) + "% of the mean pheno data")
        if(error_mean < low):
            low = error_mean
            best = m
        results.append(error_mean)
        i= i+1
    print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results), np.std(results)))
    return best

In [ ]:
best_model = eval_k_fold(xgbreg, tt_vcf, tt_pheno, 10)

In [ ]:
all_preds = [x for x in best_model.predict(ho_vcf)]
ss = sqrt(mean_squared_error(all_preds, ho_pheno))
rr = r2_score(all_preds, ho_pheno)
mm = np.mean(ho_pheno)
error_mean = ((ss/mm)*100)
print("R^2 Value of Holdout: %.2f" % rr)
print("RMSE of Holdout: %.2f" % ss)
print("Mean of Holdout: %.2f" % mm)
print("this is "+ str((ss/mm)*100) + "% of the mean pheno data")

In [ ]:
plot_x, plot_y = list(), list()

ho_pheno = ho_pheno.ravel()

for counter, i in enumerate(ho_pheno):
    if counter <= 5:
        print(counter, i, all_preds[counter])
    #zoom in a bit closer
    if(all_preds[counter] > 9):
        plot_x.append(i)
        plot_y.append(all_preds[counter])
    
#plt.plot(plot_x, plot_y, '.')
thisplot = pd.DataFrame({'Protein':plot_x, 'preds':plot_y})
#sns.jointplot(x="Oil", y="preds", data=thisplot, kind='reg' , joint_kws={'scatter_kws': {'alpha': 0.2}})
sns.regplot(x="Protein", y="preds", data=thisplot)
plt.xlabel('Real Seed Protein %')
plt.ylabel('Predicted Seed Protein %')
plt.title("XGBoost Seed Protein Prediction")
plt.show()

In [ ]:
pickle.dump(best_model, open("Protein_kfold_10_XGB.pickle.dat", "wb"))

In [ ]:
#only load if not generated in same session
best_model = pickle.load(open("Protein_kfold_10_XGB.pickle.dat", "rb"))

In [ ]:
from xgboost import plot_importance
from matplotlib import pyplot
#best_model = pickle.load(open("Oil_kfold_10_tt_from_all.pickle.dat", "rb"))
plt.figure(figsize = (20, 20))
plot_importance(best_model, max_num_features=15, importance_type='gain', height=0.3)
pyplot.show()

In [ ]:
#This function essentially returns an array of dataframe headers the length of OHE'd input SNPs for training data
#EG. It will be able to determine that feature 357310 is Gm13_17683957 but not what allele it is
#eg. feature 357309 357310 and 357311 may all be one hot encoded versions of all possible values of Gm13_17683957
#iterating through the saved OHE will by able to determine what specific allele the feature is but cannot determine
#what SNP header it belongs to. Therefore combining these two methods you can determine both allele and SNP
snp = []
imp = SimpleImputer(missing_values='./.', strategy='most_frequent')
fs_ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
x = 0
n_headers = []
le = LabelEncoder()
#while (i < 10):
for chunk in pd.read_csv("Pro_Merged_filtered.csv_train_test.csv_5pcnt.csv", chunksize=10000, index_col="Unnamed: 0"):
    chunk = chunk.T
    if 'Value' in chunk.columns:
        print("dropping value so it doesn't include that in headers")
        chunk = chunk.drop(columns=['Value'])
    headers = chunk.columns
    row_idx = chunk.index
    chunk = imp.fit_transform(chunk) #SHOULD TURN ./. into the most common for each column
    #since imputing makes a numpy array have to turn back into PD for label encoding
    chunk = pd.DataFrame(data = chunk, index = row_idx, columns = headers)
    chunk = chunk.apply(lambda col: le.fit_transform(col))
    c_headers = chunk.columns
    y = 0
    for column in chunk:
        d = (chunk[column].nunique())
        n_headers.extend([c_headers[y] for i in range(d)])
        #print(n_headers)
        #print(l)
        #n_headers.append(c_headers[y] * d)
        #print(n_headers)
        y = y + 1
    #to double check that it would indeed be one hot encoded with this amount of columns
    chunk = fs_ohe.fit_transform(chunk)
    x = x + chunk.shape[1]
    print("my X value is: " + str(x))
    print(chunk.shape)
    print("my header list is: " + str(len(n_headers)))
print(len(n_headers))

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(num=None, figsize=(10, 10), dpi=100, facecolor='w', edgecolor='k')
snp_label = []
fs = [307765,214473,469017,416842,468934,88673,468572,607723,96913,350580,61561,217747,608270,370000,480970]
scores = [358.9,353.2,293.6,285.56,278,269.5,225,169.2,163.9,160.5,146,145.1,144.4,136.8,135.6]
for jj in fs:
    jj_allele = find_snp_from_header(ohe, jj)
    this_snp = (n_headers[jj] + ' ('+str(jj_allele)+')')
    print(this_snp)
    snp_label.append(this_snp)
snp_label.reverse()
scores.reverse()
print(len(scores))
print(len(snp_label))
plt.barh(snp_label,scores)
plt.title('SNP Importance XGBoost Seed Protein')
plt.ylabel('SNP Label')
plt.xlabel('Relative F_Score (GAIN)')
plt.show()

In [ ]:
my_dict = best_model.get_booster().get_score(importance_type="gain")

In [ ]:
def rename_f_header(fn,n_headers,ohe):
    fn = fn[1:]
    fn = int(fn)
    allele = find_snp_from_header(ohe, fn)
    this_snp = (n_headers[fn] + ' ('+str(allele)+')')
    return this_snp

In [ ]:
#convert feature to actual SNP name
i = 0
new_dict = {}
for key in my_dict:
    new_key = rename_f_header(key, n_headers, ohe)
    new_dict[new_key] = my_dict[key]
    i = i + 1
    print(str(i))
    if(my_dict):
        continue
    else:
        break

In [ ]:
new_fi = pd.Series(new_dict)
print(new_fi)
df = new_fi.to_frame()
df = df.rename(columns = {0:'F_Score(GAIN)'})

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(num=None, figsize=(10, 10), dpi=100, facecolor='w', edgecolor='k')
indexes = df.nlargest(20, "F_Score(GAIN)").index
values = df.nlargest(20, "F_Score(GAIN)").values.ravel()
indexes = indexes[::-1]
values = values[::-1]
plt.barh(indexes, values)
plt.title('SNP Importance XGBoost Seed Protein')
plt.ylabel('SNP Label')
plt.xlabel('Relative F_Score (GAIN)')
plt.show()

In [ ]:
figure(num=None, figsize=(10, 10), dpi=100, facecolor='w', edgecolor='k')
indexes = df.nlargest(20, "F_Score(GAIN)").index
values = df.nlargest(20, "F_Score(GAIN)").values.ravel()
indexes = indexes[::-1]
values = values[::-1]
r_i = []
r_v = []
b_i = []
b_v = []
g_i = []
g_v = []
y_i = []
y_v = []
bl_i = []
bl_v = []
p_i = []
p_v = []
br_i = []
br_v = []
pu_i = []
pu_v = []
i = 0
while i < len(indexes):
    if('Gm16' in indexes[i]):
        r_i.append(indexes[i])
        r_v.append(values[i])
    elif('Gm20' in indexes[i]):
        b_i.append(indexes[i])
        b_v.append(values[i])
    elif('Gm15' in indexes[i]):
        g_i.append(indexes[i])
        g_v.append(values[i])
    elif('Gm03' in indexes[i]):
        y_i.append(indexes[i])
        y_v.append(values[i])
    elif('Gm08' in indexes[i]):
        p_i.append(indexes[i])
        p_v.append(values[i])
    elif('Gm04' in indexes[i]):
        br_i.append(indexes[i])
        br_v.append(values[i])
    elif('Gm13' in indexes[i]):
        pu_i.append(indexes[i])
        pu_v.append(values[i])
    else:
        bl_i.append(indexes[i])
        bl_v.append(values[i])
    i = i + 1
plt.barh(bl_i, bl_v, color="black")
plt.barh(br_i, br_v, color="brown")
plt.barh(pu_i, pu_v, color="purple")
plt.barh(g_i, g_v, color="green")
plt.barh(y_i, y_v, color="yellow")
plt.barh(p_i, p_v, color="orange")
plt.barh(b_i, b_v, color="blue")
plt.barh(r_i, r_v, color="red")
plt.title('SNP Importance XGBoost Seed Protein')
plt.ylabel('SNP Label')
plt.xlabel('Relative F_Score (GAIN)')
plt.show()

# Random Forest Model

In [ ]:
tt_vcf, ho_vcf, tt_pheno, ho_pheno = new_prep_data("Pro_Merged_filtered.csv_train_test.csv_5pcnt.csv", "Pro_Merged_filtered.csv_holdout.csv_5pcnt.csv")

In [ ]:
ohe = pickle.load(open("Pro_ohe.dat", "rb"))
tt_vcf = ohe.transform(tt_vcf)
print(tt_vcf.shape)
print(ho_vcf.shape)
ho_vcf = ohe.transform(ho_vcf)
print(ho_vcf.shape)

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=16, max_features = 'sqrt',n_jobs=-1, verbose = 1)
#trains the model, and makes the y shape as (m,) instead of (m,1)
#model.fit(X_train, y_train.ravel())

In [ ]:
best_rf_model = eval_k_fold(model, tt_vcf, tt_pheno, 10)

In [ ]:
all_preds = [x for x in best_rf_model.predict(ho_vcf)]
ss = sqrt(mean_squared_error(all_preds, ho_pheno))
rr = r2_score(all_preds, ho_pheno)
mm = np.mean(ho_pheno)
error_mean = ((ss/mm)*100)
print("R^2 Value of Holdout: %.2f" % rr)
print("RMSE of Holdout: %.2f" % ss)
print("Mean of Holdout: %.2f" % mm)
print("this is "+ str((ss/mm)*100) + "% of the mean pheno data")

In [ ]:
pickle.dump(best_rf_model, open("Protein_kfold_10_RF.pickle.dat", "wb"))

In [ ]:
#only load if not generated in same session
best_rf_model = pickle.load(open("Protein_kfold_10_RF.pickle.dat", "rb"))

# CNN Model (based upon Philipp's CNN + some changes from yield paper)

In [ ]:
tt_vcf, ho_vcf, tt_pheno, ho_pheno = new_prep_data('Pro_Merged_filtered.csv_train_test.csv_5pcnt.csv', 'Pro_Merged_filtered.csv_holdout.csv_5pcnt.csv')

In [ ]:
ohe = pickle.load(open("Pro_ohe.dat", "rb"))
tt_vcf = ohe.transform(tt_vcf)
print(tt_vcf.shape)
print(ho_vcf.shape)
ho_vcf = ohe.transform(ho_vcf)
print(ho_vcf.shape)

In [ ]:
def build_CNN_model(train_size):    
    #del model
    model = Sequential()
    model.add(Conv1D(filters=12, kernel_size=14, 
                     input_shape=(train_size, 1)))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Conv1D(filters=10, kernel_size=10, 
                     input_shape=(12, 1)))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(Conv1D(filters=8, kernel_size=8, 
                     input_shape=(10, 1)))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(48, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='linear'))
    opt = tf.keras.optimizers.Adamax(learning_rate=0.003)#, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax"


    model.compile(loss='mean_squared_error', optimizer=opt, 
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])

    print(model.summary())
    return model

In [ ]:
def eval_CNN(x, y, k):
    #model: xgboost model, should be with the best params available
    #x: input data (eg. all samples and SNPS)
    #y: labels
    #k: number of folds for cross validation
    cv = KFold(n_splits=k,shuffle=True)
  #  fig1 = plt.figure(figsize=[12,12])

   # tprs = []
   # aucs = []
    results = []
   # mean_fpr = np.linspace(0,1,100)
    low = 100
    best = []
    i = 1
    for train,test in cv.split(x,y):
        m = build_CNN_model(x.shape[1])
        print("Fold:" + str(i))
        x = x.reshape(x.shape[0], x.shape[1], 1)
        bs = ((x[train].shape[0])/50)
        bs = round(bs)
        m.fit(x[train],y[train].ravel(), validation_data=(x[test], y[test]), epochs=100, batch_size=bs)
        print("fitting done. Processing fold accuracy + checking best model")
        #predictions = [round(value) for value in y_pred]
        #sees how accurate the model was when testing the test set
        all_preds = [x for x in m.predict(x[test])]
        ss = sqrt(mean_squared_error(all_preds, y[test]))
        rr = r2_score(all_preds, y[test])
        mm = np.mean(y[test])
        error_mean = ((ss/mm)*100)
        print("R^2 Value is: " + str(rr))
        print("RMSE for dataset is:" +str(ss) + "& mean of this fold is " + str(mm))
        print("this is "+ str((ss/mm)*100) + "% of the mean pheno data")
        if(error_mean < low):
            low = error_mean
            best = m
        results.append(error_mean)
        i= i+1
        del m
    print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results), np.std(results)))
    return best

In [ ]:
#my_model = build_CNN_model()
best_model = eval_CNN(tt_vcf, tt_pheno, 10)

In [ ]:
pickle.dump(best_model, open("Oil_kfold_10_CNN.pickle.dat", "wb"))

In [ ]:
best_CNN = pickle.load(open("Oil_kfold_10_CNN.pickle.dat", "rb"))

In [ ]:
ho_vcf = ho_vcf.reshape(ho_vcf.shape[0], ho_vcf.shape[1], 1)
all_preds = [x for x in best_model.predict(ho_vcf)]
ss = sqrt(mean_squared_error(all_preds, ho_pheno))
rr = r2_score(all_preds, ho_pheno)
mm = np.mean(ho_pheno)
error_mean = ((ss/mm)*100)
print("R^2 Value of Holdout: %.2f" % rr)
print("RMSE of Holdout: %.2f" % ss)
print("Mean of Holdout: %.2f" % mm)
print("this is "+ str((ss/mm)*100) + "% of the mean pheno data")

In [ ]:
plot_x, plot_y = list(), list()

ho_pheno = ho_pheno.ravel()
print(np.isinf(ho_pheno).any())
print(np.isinf(all_preds).any())
for counter, i in enumerate(ho_pheno):
    if counter <= 5:
        print(counter, i, all_preds[counter])
    #zoom in a bit closer
    if(all_preds[counter] > 9):
        plot_x.append(i)
        plot_y.append(all_preds[counter])
    
#plt.plot(plot_x, plot_y, '.')
thisplot = pd.DataFrame({'Oil':plot_x, 'preds':plot_y})
#sns.jointplot(x="Oil", y="preds", data=thisplot, kind='reg' , joint_kws={'scatter_kws': {'alpha': 0.2}})
sns.regplot(x="Oil", y="preds", data=thisplot)
plt.xlabel('Real Seed Oil %')
plt.ylabel('Predicted Seed Oil %')
plt.title("XGBoost Seed Oil Prediction")
plt.show()

In [ ]:
from matplotlib.pyplot import figure
figure(num=None, figsize=(12, 10), dpi=100, facecolor='w', edgecolor='k')
x_ax = range(len(ho_pheno))
print(x_ax)
plt.plot(x_ax, ho_pheno, label="Actual %")
plt.plot(x_ax, all_preds, label="Predicted %") 

plt.title("Seed Oil % Predicted vs Actual")
plt.xlabel('Validation Sample')
plt.ylabel('Seed Oil Percentage')

plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
plt.figure()
plt.plot(history.history['root_mean_squared_error'])
plt.plot(history.history['val_root_mean_squared_error'])
plt.title('model accuracy')
plt.ylabel('RMSE')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
all_preds = [x[0] for x in model.predict(X_test)]
sqrt(mean_squared_error(all_preds, y_test))
r2_score(all_preds, y_test)
vcf = vcf.reshape(vcf.shape[0], vcf.shape[1], 1)
d = {'targs': [x for x in pheno], 'preds': [x[0] for x in model.predict(vcf)]}

testdf = pd.DataFrame(data=d)
testdf['minus'] = testdf['targs'] - testdf['preds']

ss = sqrt(mean_squared_error(testdf["preds"], testdf['targs']))
mm = np.mean(pheno)
print("RMSE for dataset is:" +str(ss) + "& pheno is " + str(mm))
print("this is "+ str((ss/mm)*100) + "% of the mean pheno data")

# DNN model
#### Model Based upon: Crop Yield Prediction Using Deep Neural Networks(2019)

In [4]:
tt_vcf, ho_vcf, tt_pheno, ho_pheno = new_prep_data('Pro_Merged_filtered.csv_train_test.csv_5pcnt.csv', 'Pro_Merged_filtered.csv_holdout.csv_5pcnt.csv')

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
(753,)
(753, 1)
220000
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
(189,)
(189, 1)
220000
(753, 213713)
(189, 213713)


In [5]:
ohe = pickle.load(open("Pro_ohe.dat", "rb"))
tt_vcf = ohe.transform(tt_vcf)
print(tt_vcf.shape)
print(ho_vcf.shape)
ho_vcf = ohe.transform(ho_vcf)
print(ho_vcf.shape)

(753, 620940)
(189, 213713)
(189, 620940)


In [6]:
#My own DNN model based upon paper
#del model #incase its stored a previous model
#del history #for redoing shit

#do batch size as 64
#reduce the inputs by half when you read it in
#add XGboost and RF to the one notebook
def build_DNN_model(x_len):
    model = Sequential()

    #add first input layer, with no normalization
    model.add(Dense(200, input_dim = x_len))
    model.add(Activation('relu'))
    model.add(Dropout(0.03))
    
    model.add(Dense(100))
    model.add(Activation('relu'))
    model.add(Dropout(0.02))
    
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.01))
    
    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    
    model.add(Dense(16))
    model.add(Activation('relu'))
    #add output layer
    model.add(Dense(1, activation='linear'))
    opt = tf.keras.optimizers.Adamax(learning_rate=0.003)
    model.compile(loss="mean_squared_error", optimizer=opt, metrics=[tf.keras.metrics.RootMeanSquaredError()])
    print(model.summary())
    return model




In [7]:
def eval_DNN(x, y, k):
    #model: xgboost model, should be with the best params available
    #x: input data (eg. all samples and SNPS)
    #y: labels
    #k: number of folds for cross validation
    cv = KFold(n_splits=k,shuffle=True)
  #  fig1 = plt.figure(figsize=[12,12])

   # tprs = []
   # aucs = []
    results = []
   # mean_fpr = np.linspace(0,1,100)
    low = 100
    best = []
    i = 1
    for train,test in cv.split(x,y):
        m = build_DNN_model(x.shape[1])
        print("Fold:" + str(i))
        bs = ((x[train].shape[0])/50)
        bs = round(bs)
        m.fit(x[train],y[train].ravel(), validation_data=(x[test], y[test]), epochs=100, batch_size=bs)
        print("fitting done. Processing fold accuracy + checking best model")
        #predictions = [round(value) for value in y_pred]
        #sees how accurate the model was when testing the test set
        all_preds = [x for x in m.predict(x[test])]
        ss = sqrt(mean_squared_error(all_preds, y[test]))
        rr = r2_score(all_preds, y[test])
        mm = np.mean(y[test])
        error_mean = ((ss/mm)*100)
        print("R^2 Value is: " + str(rr))
        print("RMSE for dataset is:" +str(ss) + "& mean of this fold is " + str(mm))
        print("this is "+ str((ss/mm)*100) + "% of the mean pheno data")
        if(error_mean < low):
            low = error_mean
            best = m
        results.append(error_mean)
        i= i+1
        del m
    print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results), np.std(results)))
    return best

In [8]:
#my_model = build_CNN_model()
best_DNN = eval_DNN(tt_vcf, tt_pheno, 10)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               124188200 
_________________________________________________________________
activation (Activation)      (None, 200)               0         
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               20100     
_________________________________________________________________
activation_1 (Activation)    (None, 100)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                6

677/677 [==============================] - 2s 4ms/sample - loss: 7.3742 - root_mean_squared_error: 2.7155 - val_loss: 6.3556 - val_root_mean_squared_error: 2.5210
Epoch 34/100
677/677 [==============================] - 2s 4ms/sample - loss: 7.6718 - root_mean_squared_error: 2.7698 - val_loss: 5.6687 - val_root_mean_squared_error: 2.3809
Epoch 35/100
677/677 [==============================] - 2s 4ms/sample - loss: 6.9272 - root_mean_squared_error: 2.6319 - val_loss: 5.5979 - val_root_mean_squared_error: 2.3660
Epoch 36/100
677/677 [==============================] - 2s 4ms/sample - loss: 7.9470 - root_mean_squared_error: 2.8190 - val_loss: 5.8490 - val_root_mean_squared_error: 2.4185
Epoch 37/100
677/677 [==============================] - 2s 4ms/sample - loss: 6.5769 - root_mean_squared_error: 2.5645 - val_loss: 6.0165 - val_root_mean_squared_error: 2.4529
Epoch 38/100
677/677 [==============================] - 3s 4ms/sample - loss: 6.9091 - root_mean_squared_error: 2.6285 - val_loss: 5.

677/677 [==============================] - 3s 4ms/sample - loss: 2.5479 - root_mean_squared_error: 1.5962 - val_loss: 6.7730 - val_root_mean_squared_error: 2.6025
Epoch 80/100
677/677 [==============================] - 3s 4ms/sample - loss: 2.6843 - root_mean_squared_error: 1.6384 - val_loss: 5.5559 - val_root_mean_squared_error: 2.3571
Epoch 81/100
677/677 [==============================] - 2s 4ms/sample - loss: 3.0614 - root_mean_squared_error: 1.7497 - val_loss: 7.9882 - val_root_mean_squared_error: 2.8263
Epoch 82/100
677/677 [==============================] - 2s 4ms/sample - loss: 2.4620 - root_mean_squared_error: 1.5691 - val_loss: 6.4406 - val_root_mean_squared_error: 2.5378
Epoch 83/100
677/677 [==============================] - 2s 4ms/sample - loss: 3.2925 - root_mean_squared_error: 1.8145 - val_loss: 6.8125 - val_root_mean_squared_error: 2.6101
Epoch 84/100
677/677 [==============================] - 2s 4ms/sample - loss: 2.3182 - root_mean_squared_error: 1.5226 - val_loss: 6.

677/677 [==============================] - 2s 4ms/sample - loss: 60.8029 - root_mean_squared_error: 7.7976 - val_loss: 17.5687 - val_root_mean_squared_error: 4.1915
Epoch 11/100
677/677 [==============================] - 2s 4ms/sample - loss: 58.4784 - root_mean_squared_error: 7.6471 - val_loss: 26.9720 - val_root_mean_squared_error: 5.1935
Epoch 12/100
677/677 [==============================] - 2s 4ms/sample - loss: 43.2538 - root_mean_squared_error: 6.5768 - val_loss: 29.0390 - val_root_mean_squared_error: 5.3888
Epoch 13/100
677/677 [==============================] - 2s 4ms/sample - loss: 35.9471 - root_mean_squared_error: 5.9956 - val_loss: 33.0134 - val_root_mean_squared_error: 5.7457
Epoch 14/100
677/677 [==============================] - 3s 4ms/sample - loss: 27.3686 - root_mean_squared_error: 5.2315 - val_loss: 12.2757 - val_root_mean_squared_error: 3.5037
Epoch 15/100
677/677 [==============================] - 3s 4ms/sample - loss: 28.4022 - root_mean_squared_error: 5.3294 - v

677/677 [==============================] - 2s 4ms/sample - loss: 5.1673 - root_mean_squared_error: 2.2732 - val_loss: 14.5998 - val_root_mean_squared_error: 3.8210
Epoch 57/100
677/677 [==============================] - 2s 4ms/sample - loss: 4.9578 - root_mean_squared_error: 2.2266 - val_loss: 7.3293 - val_root_mean_squared_error: 2.7073
Epoch 58/100
677/677 [==============================] - 2s 4ms/sample - loss: 4.9930 - root_mean_squared_error: 2.2345 - val_loss: 5.9636 - val_root_mean_squared_error: 2.4420
Epoch 59/100
677/677 [==============================] - 2s 4ms/sample - loss: 4.2375 - root_mean_squared_error: 2.0585 - val_loss: 5.6023 - val_root_mean_squared_error: 2.3669
Epoch 60/100
677/677 [==============================] - 2s 4ms/sample - loss: 5.0072 - root_mean_squared_error: 2.2377 - val_loss: 8.0985 - val_root_mean_squared_error: 2.8458
Epoch 61/100
677/677 [==============================] - 2s 4ms/sample - loss: 4.3672 - root_mean_squared_error: 2.0898 - val_loss: 7

...

678/678 [==============================] - 3s 4ms/sample - loss: 844.7579 - root_mean_squared_error: 29.0647 - val_loss: 327.6957 - val_root_mean_squared_error: 18.1024
Epoch 4/100
678/678 [==============================] - 3s 4ms/sample - loss: 302.4061 - root_mean_squared_error: 17.3898 - val_loss: 219.6721 - val_root_mean_squared_error: 14.8213
Epoch 5/100
678/678 [==============================] - 3s 4ms/sample - loss: 94.0099 - root_mean_squared_error: 9.6959 - val_loss: 77.6746 - val_root_mean_squared_error: 8.8133
Epoch 6/100
678/678 [==============================] - 3s 4ms/sample - loss: 61.5132 - root_mean_squared_error: 7.8430 - val_loss: 158.5557 - val_root_mean_squared_error: 12.5919
Epoch 7/100
678/678 [==============================] - 2s 4ms/sample - loss: 42.3776 - root_mean_squared_error: 6.5098 - val_loss: 94.6783 - val_root_mean_squared_error: 9.7303
Epoch 8/100
678/678 [==============================] - 2s 4ms/sample - loss: 32.0874 - root_mean_squared_error: 5.664

678/678 [==============================] - 2s 4ms/sample - loss: 3.9165 - root_mean_squared_error: 1.9790 - val_loss: 6.8578 - val_root_mean_squared_error: 2.6187
Epoch 50/100
678/678 [==============================] - 3s 4ms/sample - loss: 3.8734 - root_mean_squared_error: 1.9681 - val_loss: 7.6756 - val_root_mean_squared_error: 2.7705
Epoch 51/100
678/678 [==============================] - 2s 4ms/sample - loss: 3.7435 - root_mean_squared_error: 1.9348 - val_loss: 7.2389 - val_root_mean_squared_error: 2.6905
Epoch 52/100
678/678 [==============================] - 2s 4ms/sample - loss: 3.2476 - root_mean_squared_error: 1.8021 - val_loss: 6.6536 - val_root_mean_squared_error: 2.5795
Epoch 53/100
678/678 [==============================] - 2s 4ms/sample - loss: 3.2022 - root_mean_squared_error: 1.7895 - val_loss: 8.4307 - val_root_mean_squared_error: 2.9036
Epoch 54/100
678/678 [==============================] - 2s 4ms/sample - loss: 2.8966 - root_mean_squared_error: 1.7019 - val_loss: 8.

678/678 [==============================] - 3s 4ms/sample - loss: 1.8095 - root_mean_squared_error: 1.3452 - val_loss: 9.3351 - val_root_mean_squared_error: 3.0553
Epoch 96/100
678/678 [==============================] - 3s 4ms/sample - loss: 2.2849 - root_mean_squared_error: 1.5116 - val_loss: 6.4615 - val_root_mean_squared_error: 2.5419
Epoch 97/100
678/678 [==============================] - 2s 4ms/sample - loss: 1.7958 - root_mean_squared_error: 1.3401 - val_loss: 6.4085 - val_root_mean_squared_error: 2.5315
Epoch 98/100
678/678 [==============================] - 2s 4ms/sample - loss: 1.9441 - root_mean_squared_error: 1.3943 - val_loss: 6.5587 - val_root_mean_squared_error: 2.5610
Epoch 99/100
678/678 [==============================] - 3s 4ms/sample - loss: 1.4556 - root_mean_squared_error: 1.2065 - val_loss: 7.1276 - val_root_mean_squared_error: 2.6698
Epoch 100/100
678/678 [==============================] - 2s 4ms/sample - loss: 1.3179 - root_mean_squared_error: 1.1480 - val_loss: 7

678/678 [==============================] - 3s 4ms/sample - loss: 10.0706 - root_mean_squared_error: 3.1734 - val_loss: 9.5423 - val_root_mean_squared_error: 3.0891
Epoch 27/100
678/678 [==============================] - 3s 4ms/sample - loss: 11.3128 - root_mean_squared_error: 3.3634 - val_loss: 9.0208 - val_root_mean_squared_error: 3.0035
Epoch 28/100
678/678 [==============================] - 3s 4ms/sample - loss: 10.7205 - root_mean_squared_error: 3.2742 - val_loss: 17.1665 - val_root_mean_squared_error: 4.1432
Epoch 29/100
678/678 [==============================] - 3s 4ms/sample - loss: 11.1139 - root_mean_squared_error: 3.3338 - val_loss: 10.3250 - val_root_mean_squared_error: 3.2132
Epoch 30/100
678/678 [==============================] - 3s 4ms/sample - loss: 10.4307 - root_mean_squared_error: 3.2297 - val_loss: 10.1341 - val_root_mean_squared_error: 3.1834
Epoch 31/100
678/678 [==============================] - 3s 4ms/sample - loss: 11.2381 - root_mean_squared_error: 3.3523 - val

678/678 [==============================] - 3s 4ms/sample - loss: 3.9673 - root_mean_squared_error: 1.9918 - val_loss: 10.3240 - val_root_mean_squared_error: 3.2131
Epoch 73/100
678/678 [==============================] - 3s 4ms/sample - loss: 3.4429 - root_mean_squared_error: 1.8555 - val_loss: 9.5923 - val_root_mean_squared_error: 3.0971
Epoch 74/100
678/678 [==============================] - 3s 4ms/sample - loss: 4.0308 - root_mean_squared_error: 2.0077 - val_loss: 10.2618 - val_root_mean_squared_error: 3.2034
Epoch 75/100
678/678 [==============================] - 3s 4ms/sample - loss: 3.3931 - root_mean_squared_error: 1.8420 - val_loss: 11.0780 - val_root_mean_squared_error: 3.3284
Epoch 76/100
678/678 [==============================] - 3s 4ms/sample - loss: 2.5857 - root_mean_squared_error: 1.6080 - val_loss: 11.6573 - val_root_mean_squared_error: 3.4143
Epoch 77/100
678/678 [==============================] - 3s 4ms/sample - loss: 2.7155 - root_mean_squared_error: 1.6479 - val_loss

In [9]:
#ho_vcf = ho_vcf.reshape(ho_vcf.shape[0], ho_vcf.shape[1], 1)
all_preds = [x for x in best_DNN.predict(ho_vcf)]
ss = sqrt(mean_squared_error(all_preds, ho_pheno))
rr = r2_score(all_preds, ho_pheno)
mm = np.mean(ho_pheno)
error_mean = ((ss/mm)*100)
print("R^2 Value of Holdout: %.2f" % rr)
print("RMSE of Holdout: %.2f" % ss)
print("Mean of Holdout: %.2f" % mm)
print("this is "+ str((ss/mm)*100) + "% of the mean pheno data")

R^2 Value of Holdout: -0.94
RMSE of Holdout: 3.11
Mean of Holdout: 44.92
this is 6.913757869520916% of the mean pheno data


In [ ]:
plot_x, plot_y = list(), list()

ho_pheno = ho_pheno.ravel()

for counter, i in enumerate(ho_pheno):
    if counter <= 5:
        print(counter, i, all_preds[counter])
    #zoom in a bit closer
    if(all_preds[counter] > 9):
        plot_x.append(i)
        plot_y.append(all_preds[counter])
    
#plt.plot(plot_x, plot_y, '.')
thisplot = pd.DataFrame({'Oil':plot_x, 'preds':plot_y})
#sns.jointplot(x="Oil", y="preds", data=thisplot, kind='reg' , joint_kws={'scatter_kws': {'alpha': 0.2}})
sns.regplot(x="Oil", y="preds", data=thisplot)
plt.xlabel('Real Seed Oil %')
plt.ylabel('Predicted Seed Oil %')
plt.title("DNN Seed Oil Prediction")
plt.show()

In [ ]:
from matplotlib.pyplot import figure
figure(num=None, figsize=(12, 10), dpi=100, facecolor='w', edgecolor='k')
x_ax = range(len(ho_pheno))
print(x_ax)
plt.plot(x_ax, ho_pheno, label="Actual %")
plt.plot(x_ax, all_preds, label="Predicted %") 

plt.title("Seed Oil % Predicted vs Actual")
plt.xlabel('Validation Sample')
plt.ylabel('Seed Oil Percentage')

plt.legend()
plt.show()

In [ ]:
pickle.dump(best_DNN, open("Oil_kfold_10_DNN.pickle.dat", "wb"))

In [ ]:
best_DNN = pickle.load(open("Oil_kfold_10_DNN.pickle.dat", "rb"))

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
plt.figure()
plt.plot(history.history['root_mean_squared_error'])
plt.plot(history.history['val_root_mean_squared_error'])
plt.title('model accuracy')
plt.ylabel('RMSE')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
y_test=ho_pheno
x_ax = range(len(y_test))
print(x_ax)
plt.plot(x_ax, y_test, label="original")
plt.plot(x_ax, all_preds, label="predicted") 

plt.title("Test and predicted data")

plt.legend()
plt.show()

i = 0
interval = 65
while(i<len(y_test)):
    i = i + interval
    if(i>len(y_test)):
        my_range = range(i-interval,len(y_test))
    else:
        my_range = range(i-interval,i)
    plt.plot(my_range, y_test[my_range], label="original")
    plt.plot(my_range, all_preds[my_range], label="predicted") 

    plt.title("Test and predicted data")

    plt.legend()
    plt.show()

In [ ]:
d = {'targs': [x for x in pheno], 'preds': [x[0] for x in model.predict(vcf)]}

testdf = pd.DataFrame(data=d)
testdf['minus'] = testdf['targs'] - testdf['preds']

In [ ]:
ss = sqrt(mean_squared_error(testdf["preds"], testdf['targs']))
mm = np.mean(pheno)
print("RMSE for dataset is:" +str(ss) + "& pheno is " + str(mm))
print("this is "+ str((ss/mm)*100) + "% of the mean pheno data")